<a href="https://colab.research.google.com/github/Honsl/Llama2-Prompt-Engineering/blob/main/Prompt_Replicate_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

To run this notebook, you’ll need to create a [Replicate](https://replicate.com) account and install the Replicate python client.

In [ ]:
# install replicate client
!pip install replicate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.0 MB/s eta 0:00:00


# Run and then paste in your API code

In [ ]:
# get a token: https://replicate.com/account
from getpass import getpass
import os
from google.colab import output


REPLICATE_API_TOKEN = getpass()
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

··········


Functions to manage the length of prompts

In [ ]:
#Format the prompt message for Llama2
def createMessage(messages):
  message = ""
  for x in messages:
    #if the text is from the user surround with INST
    if x["isUser"]:
      message+="[INST]"+x["text"]+"[/INST]"
    #else it is a message from Llama2
    else:
      message+=x["text"]
  return message

#Delete the oldest message
def shrinkMessage(messages):
  del messages[0]

#Get the current word count of the prompt
def messagesSize(messages):
  messageLength = 0
  for x in messages:
    messageLength+=len(x["text"].split())
  return messageLength

## Responses can be slow - as at times it needs to boot up an instance to reply (3-5 mins).

**2023-12-04:**
**V1.1:** Fixed sent prompts having duplicate information

**2023-12-06:**
**V1.2:** Message are now stored in Dictionary and remove old messages if prompt is over max tokens

---


**TODO:**




In [ ]:
import replicate
#Speak in French.
system_prompt = """
You are a server at a coffee shop.
Start conversation with:"Hi, good morning. My name is Allan. What would you like to order?".
keep responses short.
Do not make suggestions to me, unless I ask for suggestions.
Do not offer other items.
Do not ask for my name.
You only sell basic coffee and dougnuts.
coffee is $1.50 and espresso is $2.50. The coffee sizes are small,medium and large.
The dougnnuts you sell are glazed and chocolate and they cost $1.00.
Do not list the products you sell and their prices, unless I ask you to list them.
Do not ask what temperature the coffee should be.
The coffee is served hot.
"""
#Max number of tokens(words) that can be sent as the prompt - DO NOT CHANGE
MAX_TOKENS = 3000

#Changable settings for Llama2
max_new_tokens = 128 #default 128
min_new_tokens = -1 #default -1
temperature = 0.1 #default 0.75
top_p = 0.9 #default 0.9
top_k = 50 #default 50
debug = False

messageHistory = [
     {"text": "Hi",
      "isUser": True}]
new_prompt = 'empty'
#continue to prompt the user untill empty string
while(new_prompt!=''):
  iterator = replicate.run(
  "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
  input={
                              "prompt": createMessage(messageHistory),
                              "system_prompt": system_prompt,
                              "max_new_tokens": max_new_tokens,
                              "min_new_tokens": min_new_tokens,
                              "temperature": temperature,
                              "top_p": top_p,
                              "top_k": top_k,
                              "debug": debug
                          })
#output the message from Llama2
  message = ""
  count = 0
  for text in iterator:
    output.clear()
    count+=1
    if(count>=20):
      message+='\n'
      count=0
    message+=text
    print(message)
#ask for a new prompt from the user
  new_prompt = input('Enter next prompt: ')
#save the response from Llama2
  messageHistory.append({
      "text": message,
      "isUser": False
  });
#save the new prompt from the user
  messageHistory.append({
      "text": new_prompt,
      "isUser": True
  });
#make sure the prompt messages does not go over the max # of tokens
#and remove the oldest message if it is over
  while(messagesSize(messageHistory)>MAX_TOKENS):
    shrinkMessage(messageHistory)







 Sure thing! That'll be $2.50 please. Would you like the
 coffee hot or icy?
